In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import requests
import matplotlib.cm as cm

import os
from dotenv import load_dotenv
import json
import logging
#os.makedirs('./check', exist_ok=True)
# logging.basicConfig(
#     filename='./check/data_processing.log',
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(message)s'
# )
#logging.basicConfig(filename='./check/debug.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_visits(api_url, start_date, end_date):
    try:
        response = requests.get(f"{api_url}/visits?", params={'begin': start_date, 'end': end_date})
        response.raise_for_status()
        
        # Печать JSON-ответа
        visits_data = response.json()  # Получаем данные из ответа
        file_path = './check/http_visits.json'
        # with open(file_path, 'w') as f:
        #     json.dump(visits_data, f, indent=4, default=str)  # Используем indent для удобочитаемости

        # print(f"Data saved to {file_path}")
        return visits_data         # Возвращаем данные
        
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None
    except Exception as err:
        print(f"Other error occurred: {err}")
        return None

def fetch_registrations(api_url, start_date, end_date):
    try:
        response = requests.get(f"{api_url}/registrations?", params={'begin': start_date, 'end': end_date})
        response.raise_for_status()
        registration_data = response.json()  # Получаем данные из ответа
        file_path = './check/http_registrations.json'
        # with open(file_path, 'w') as f:
        #     json.dump(registration_data, f, indent=4, default=str)  # Используем indent для удобочитаемости

        # print(f"Data saved to {file_path}")        # Печатаем полученные данные
        
        return registration_data
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None
    except Exception as err:
        print(f"Other error occurred: {err}")
        return None

load_dotenv()

api_url = os.getenv("API_URL")
start_date = os.getenv("DATE_BEGIN")
end_date = os.getenv("DATE_END")

visits_data = fetch_visits(api_url, start_date, end_date)
registrations_data = fetch_registrations(api_url, start_date, end_date)

if visits_data is None or registrations_data is None:
    print("Failed to fetch data. Exiting.")
    exit(1)
# Convert data to DataFrames and prepare it
# Преобразуем данные в DataFrames
df_reg = pd.DataFrame(registrations_data)
df_vis = pd.DataFrame(visits_data)

# Фильтруем визиты, исключая ботов
df_vis = df_vis[~df_vis['user_agent'].str.contains("bot", case=False, na=False)]
#logging.info(f"Filtered Visits (no bots):\n{df_vis.head()}")

# Преобразование столбца datetime
df_reg['datetime'] = pd.to_datetime(df_reg['datetime'])
df_vis['datetime'] = pd.to_datetime(df_vis['datetime'])
#logging.info("Datetime conversion done.")

# Оставляем только последний визит для каждого visit_id
df_vis_last = df_vis.sort_values('datetime').groupby('visit_id').last().reset_index()

# Заполняем отсутствующие значения
df_vis_last = df_vis_last.fillna(0)
df_reg = df_reg.fillna(0)

# Группируем последние визиты по дате и платформе
df_vis_grouped = df_vis_last.groupby([df_vis_last['datetime'].dt.date, 'platform']).size().reset_index(name='visits')
#logging.info(f"Grouped Last Visits DataFrame:\n{df_vis_grouped}")

# Группируем регистрации по дате и платформе
df_reg_grouped = df_reg.groupby([df_reg['datetime'].dt.date, 'platform']).size().reset_index(name='registrations')
#logging.info(f"Grouped Registrations DataFrame:\n{df_reg_grouped}")

# Объединяем DataFrames
merged_df = pd.merge(df_vis_grouped, df_reg_grouped, on=['datetime', 'platform'], how='left')
#logging.info(f"Merged DataFrame (before filling NaN in registrations):\n{merged_df}")

# Заполняем отсутствующие значения в регистрациях
merged_df['registrations'] = merged_df['registrations'].fillna(0)

# Вычисляем конверсию
merged_df['conversion'] = merged_df.apply(
    lambda row: (row['registrations'] / row['visits'] * 100) if row['visits'] > 0 else 0,
    axis=1
)
#logging.info(f"Merged DataFrame (after conversion calculation):\n{merged_df}")

# Преобразуем дату в миллисекунды для JSON
merged_df['date_group'] = pd.to_datetime(merged_df['datetime']).astype(int) // 10**6
#logging.info(f"Final DataFrame with milliseconds:\n{merged_df[['datetime', 'date_group']]}")
merged_df['conversion'] = merged_df['conversion'].apply(lambda x: round(x, 10))
# Создаем финальный JSON
final_json = {
    "date_group": {str(i): int(v) for i, v in enumerate(merged_df['date_group'])},
    "platform": {str(i): p for i, p in enumerate(merged_df['platform'])},
    "visits": {str(i): int(v) for i, v in enumerate(merged_df['visits'])},
    "registrations": {str(i): int(v) for i, v in enumerate(merged_df['registrations'])},
    "conversion": {str(i): v for i, v in enumerate(merged_df['conversion'])}
}

# Save to JSON
with open('conversion.json', 'w') as f:
    json.dump(final_json, f, indent=4)
#logging.info("Saved final JSON to './check/conversion.json'")

ads_df = pd.read_csv('ads.csv')
#ads_df['date'] = pd.to_datetime(ads_df['date'], errors='coerce')  # Преобразуем в datetime, обрабатываем некорректные значения
#ads_df['date'] = ads_df['date'].fillna(pd.Timestamp(0))  # Заменяем NaT на дефолтное значение
#ads_df['date'] = ads_df['date'].astype(int) // 10**6  # Преобразуем в Unix timestamp в миллисекундах
ads_df['date'] = pd.to_datetime(ads_df['date']).dt.strftime('%Y-%m-%d')
ads_df['date'] = pd.to_datetime(ads_df['date']).astype(int) // 10 ** 6
#logging.info(f"Dates from ads_df (formatted to YYYY-MM-DD):\n{ads_df['date'].tolist()}")
ads_dates_json = {
    "date_group": {str(i): int(v) for i, v in enumerate(ads_df['date'])}  # Даты рекламы после преобразования
}

# Сохраняем JSON с датами из ads_df
# with open('./check/ads_dates.json', 'w') as f:
#     json.dump(ads_dates_json, f, indent=4)
# Группируем данные рекламы по дате
ads_cost = ads_df.groupby('date').agg({'cost': 'sum'}).reset_index().fillna(0)
ads_campaign = ads_df.groupby('date').agg({'utm_campaign': 'first'}).reset_index().fillna('none')

# Выполняем левое объединение данных рекламы с основными данными по дате
# Это позволит сопоставить данные рекламы только для дат, которые есть в merged_df
ads_merged = pd.merge(merged_df[['date_group']], ads_cost, how='left', left_on='date_group', right_on='date')

# Теперь сопоставляем utm_campaign по дате
ads_merged = pd.merge(ads_merged, ads_campaign[['date', 'utm_campaign']], how='left', left_on='date_group', right_on='date')

# Заменяем NaN на 0 для столбца cost и 'none' для utm_campaign, если данных нет для какой-то даты
ads_merged['cost'] = ads_merged['cost'].fillna(0)
ads_merged['utm_campaign'] = ads_merged['utm_campaign'].fillna('none')

# Создаем финальный JSON для рекламы (второй JSON)
final_ads_json = {
    "date_group": {str(i): int(v) for i, v in enumerate(merged_df['date_group'])},  # Используем даты из merged_df
    "visits": {str(i): int(v) for i, v in enumerate(merged_df['visits'])},
    "registrations": {str(i): int(v) for i, v in enumerate(merged_df['registrations'])},
    "cost": {str(i): int(ads_merged.iloc[i]['cost']) for i in range(len(merged_df))},  # Заполняем затраты из рекламы
    "utm_campaign": {str(i): ads_merged.iloc[i]['utm_campaign'] for i in range(len(merged_df))}  # Заполняем utm_campaign из рекламы
}

# Сохраняем финальный JSON для рекламы
with open('ads.json', 'w') as f:
    json.dump(final_ads_json, f, indent=4)



# Подготовка данных с новыми именами
summary_df = pd.DataFrame(final_json)  # Переименована переменная merged_df
ads_summary_df = pd.DataFrame(final_ads_json)  # Переименована переменная ads_merged


# Преобразование даты из UNIX времени в человекочитаемый формат
summary_df['date_group'] = pd.to_datetime(summary_df['date_group'], unit='ms')

# Удаляем строки с рекламными кампаниями, где 'utm_campaign' равен None
ads_summary_df = ads_summary_df[ads_summary_df['utm_campaign'].notna()]
summary_df.head()
# Создание директории для графиков, если она не существует
if not os.path.exists('charts'):
    os.makedirs('charts')

# Функция для сохранения графиков
def save_plot(plt, filename):
    # Генерируем полный путь для сохранения файла
    path = os.path.join('charts', filename)
    
    # Выводим путь для проверки
    print(f"Saving plot to: {path}")
    
    # Сохраняем график
    plt.savefig(path, bbox_inches='tight', dpi=300)
    
    # Закрываем график
    plt.close()


# Функция для аннотации всех локальных вершин и низин
def annotate_peaks_troughs(ax, x_data, y_data):
    # Проход по точкам и нахождение локальных максимумов и минимумов
    for i in range(1, len(y_data) - 1):
        # Локальный максимум
        if y_data[i] > y_data[i - 1] and y_data[i] > y_data[i + 1]:
            ax.annotate(f'{y_data[i]:.2f}%', xy=(x_data[i], y_data[i]),
                        xytext=(0, 5), textcoords='offset points', color='green', fontsize=10, ha='center')
        # Локальный минимум
        if y_data[i] < y_data[i - 1] and y_data[i] < y_data[i + 1]:
            ax.annotate(f'{y_data[i]:.2f}%', xy=(x_data[i], y_data[i]),
                        xytext=(0, -10), textcoords='offset points', color='red', fontsize=10, ha='center')


# 1 Итоговые визиты
merged_df['date_group'] = pd.to_datetime(merged_df['date_group'], unit='ms')
aggregated_df = merged_df.groupby('date_group', as_index=False)['visits'].sum()
# Агрегация данных: группируем по дате и суммируем визиты
aggregated_df['x_pos'] = range(len(aggregated_df))  # Начинаем с 0, а не с 1

plt.figure(figsize=(48, 24))

# Построение столбикового графика с явным выравниванием по центру
bars = plt.bar(aggregated_df['x_pos'], aggregated_df['visits'], color='skyblue', align='center')

# Добавление значений над каждым столбцом
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')  # Значение над столбцом

# Добавление заголовка и подписей осей
plt.title('Итоговые визиты')
plt.xlabel('Дата')
plt.ylabel('Количество визитов')

# Замена числовых индексов на реальные даты (настраиваем метки на основе позиций столбиков)
plt.xticks(aggregated_df['x_pos'], aggregated_df['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right')

# Сетка по оси Y
plt.grid(axis='y')

# Установка пределов по оси Y
plt.ylim(0, aggregated_df['visits'].max() * 1.1)  # Установите максимум на 10% больше максимума

# Сохранение графика
save_plot(plt, 'Итоговые визиты.png')

# Отображение графика
plt.show()





# 2 график Итоговые визиты с разбивкой по платформам: web, android, ios
# Преобразуем данные в формат для построения столбчатой диаграммы

pivot_data = summary_df.pivot_table(index='date_group', columns='platform', values='visits', aggfunc='sum')

# Построение столбчатой диаграммы с накоплением
pivot_data.plot(kind='bar', stacked=True, figsize=(48, 24), color=plt.cm.Paired.colors)

# Настройки графика
plt.title('Итоговые визиты по платформам (с накоплением)')
plt.xlabel('Дата')
plt.ylabel('Количество визитов')
plt.xticks(aggregated_df['x_pos'], aggregated_df['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right')
plt.xticks(rotation=45)  # Поворачиваем метки оси X для лучшей читаемости
plt.grid(axis='y')  # Сетка только по оси Y
plt.grid(axis='x')
plt.legend(title='Платформы')

# Сохранение графика
save_plot(plt, 'Итоговые визиты с разбивкой по платформам.png')






# 3. Итоговые регистрации

# Создание фигуры
plt.figure(figsize=(48, 24))

# Преобразование столбца 'date_group' в формат datetime
merged_df['date_group'] = pd.to_datetime(merged_df['date_group'], unit='ms')

# Агрегация данных: группируем по дате и суммируем регистрации
aggregated_df = merged_df.groupby('date_group', as_index=False)['registrations'].sum()

# Построение столбикового графика
bars = plt.bar(aggregated_df['date_group'], aggregated_df['registrations'], color='skyblue', align='center')

# Добавление значений над каждым столбцом
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')  # Значение над столбцом

# Настройки графика
plt.title('Итоговые регистрации')
plt.xlabel('Дата')
plt.ylabel('Количество регистраций')

# Установка меток оси X на все даты
plt.xticks(aggregated_df['date_group'], aggregated_df['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right')

# Сетка по оси Y
plt.grid(axis='y')

# Установка пределов по оси Y
plt.ylim(0, aggregated_df['registrations'].max() * 1.1)  # Установите максимум на 10% больше максимума

# Сохранение графика
save_plot(plt, 'Итоговые регистрации.png')

# Отображение графика
plt.show()



# # 4. Итоговые регистрации с разбивкой по платформам (линейный график)
# Преобразуем данные в формат для построения столбчатой диаграммы
pivot_data_registrations = summary_df.pivot_table(index='date_group', columns='platform', values='registrations', aggfunc='sum')

# Построение столбчатой диаграммы с накоплением для регистраций
pivot_data_registrations.plot(kind='bar', stacked=True, figsize=(48, 24), color=plt.cm.Paired.colors)

# Настройки графика
plt.title('Итоговые регистрации по платформам (с накоплением)')
plt.xlabel('Дата')
plt.ylabel('Количество регистраций')

# Установка меток оси X на все даты
plt.xticks(ticks=range(len(pivot_data_registrations.index)), labels=pivot_data_registrations.index.strftime('%Y-%m-%d'), rotation=45, ha='right')

# Сетка по оси Y
plt.grid(axis='y')  # Сетка только по оси Y
plt.grid(axis='x')  # Сетка по оси X

plt.legend(title='Платформы')
# Сохранение графика
save_plot(plt, 'Итоговые регистрации с разбивкой по платформе.png')

# Отображение графика
plt.show()





# # 5. Конверсия по каждой платформе

# Создание фигуры с тремя подграфиками (по количеству платформ)
fig, axs = plt.subplots(3, 1, figsize=(48, 24), sharex=True)

# Платформы для построения
platforms = ['web', 'android', 'ios']





for i, platform in enumerate(platforms):
    # Фильтрация данных по текущей платформе
    platform_data = merged_df[merged_df['platform'] == platform]
    
    # Данные для построения графика
    x_data = platform_data['datetime'].values
    y_data = platform_data['conversion'].values
    
    # Построение линии для платформы
    axs[i].plot(x_data, y_data, label=platform, color='b')
    
    # Добавление аннотаций на всех вершинах и низинах
    annotate_peaks_troughs(axs[i], x_data, y_data)
    
    # Настройки для каждого графика
    axs[i].set_title(f'Conversion Rate for {platform.capitalize()}')
    axs[i].set_ylabel('Conversion Rate (%)')
    axs[i].legend(loc='upper right')
    axs[i].grid(True)

# Общая настройка для оси X (дата)
axs[2].set_xlabel('Date')
plt.xticks(rotation=45)


save_plot(plt, 'Конверсия по каждой платформе.png')




# # 6. Средняя конверсия
# Построение графика
# Суммирование визитов и регистраций по всем платформам для каждой даты
merged_df_grouped = merged_df.groupby('datetime').agg({
    'visits': 'sum',
    'registrations': 'sum'
}).reset_index()

# Пересчет конверсии
merged_df_grouped['conversion'] = (merged_df_grouped['registrations'] / merged_df_grouped['visits'] * 100).fillna(0)



# Построение единого графика
plt.figure(figsize=(48, 24))

# Данные для построения единого графика
x_data = merged_df_grouped['datetime'].values
y_data = merged_df_grouped['conversion'].values

# Построение линии для суммарной конверсии
plt.plot(x_data, y_data, label='Total Conversion', color='b')

# Добавление аннотаций для всех вершин и низин
annotate_peaks_troughs(plt.gca(), x_data, y_data)

# Настройки графика
plt.title('Overall Conversion Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Conversion Rate (%)')
plt.xticks(rotation=45)
plt.legend(title='Conversion')

# Сохранение графика
save_plot(plt, 'Средняя конверсия.png')




# 7. Стоимости реклам (упрощенный график без None)
# Исключаем кампанию с названием 'none'
# Настройка логирования

# Исключаем кампанию с названием 'none'
filtered_ads_df = ads_summary_df[ads_summary_df['utm_campaign'] != 'none']

# Сортируем данные по дате
filtered_ads_df = filtered_ads_df.sort_values(by='date_group')  # 'date_group' - название столбца с датами

# Преобразуем 'date_group' в datetime для правильного отображения
filtered_ads_df['date_group'] = pd.to_datetime(filtered_ads_df['date_group'], unit='ms')  # Если даты в миллисекундах

# Получаем список уникальных кампаний
campaigns = filtered_ads_df['utm_campaign'].unique()

# Устанавливаем цвета для каждой кампании
colors = cm.get_cmap('tab10', len(campaigns))  # Используем палитру с 10 цветами

# Функция для нахождения локальных максимумов и минимумов
def find_peaks_and_troughs(x_data, y_data):
    peaks = []
    troughs = []
    
    for i in range(1, len(y_data) - 1):
        # Проверяем на максимум
        if y_data[i] > y_data[i - 1] and y_data[i] > y_data[i + 1]:
            peaks.append(i)  # Индекс локального максимума
        # Проверяем на минимум
        elif y_data[i] < y_data[i - 1] and y_data[i] < y_data[i + 1]:
            troughs.append(i)  # Индекс локального минимума

        # Дополнительная проверка для случаев, когда значения одинаковы
        elif y_data[i] == y_data[i - 1] and y_data[i] > y_data[i + 1]:
            peaks.append(i)  # Добавляем, если текущая равна предыдущей, но больше следующей
        elif y_data[i] == y_data[i + 1] and y_data[i] < y_data[i - 1]:
            troughs.append(i)  # Добавляем, если текущая равна следующей, но меньше предыдущей

    # Логируем данные для отладки
    logging.info(f"Data: {y_data}, Peaks: {peaks}, Troughs: {troughs}")
    return peaks, troughs

# Построение линейного графика
plt.figure(figsize=(48, 24))  # Размер графика

# Переменные для хранения данных
prev_campaign = None
x_data, y_data = [], []
color_index = 0  # Для переключения цветов

# Проход по каждой строке и построение линий с разными цветами
for i, row in filtered_ads_df.iterrows():
    campaign = row['utm_campaign']
    
    # Если текущая кампания отличается от предыдущей
    if campaign != prev_campaign:
        # Если есть данные для предыдущей кампании, рисуем линию
        if prev_campaign is not None and len(x_data) > 0 and len(y_data) > 0:
            plt.plot(x_data, y_data, label=prev_campaign, color=colors(color_index), linewidth=2)  # Установите желаемую толщину
            
            # Находим локальные максимумы и минимумы
            peaks, troughs = find_peaks_and_troughs(x_data, y_data)
            logging.info(f"Campaign: {prev_campaign}, Peaks: {peaks}, Troughs: {troughs}")  # Логирование информации
            
            # Аннотируем локальные максимумы и минимумы
            last_peak_value = None  # Храним последнее аннотированное значение пика
            last_trough_value = None  # Храним последнее аннотированное значение минимума
            
            # Объединяем индексы пиков и низин
            for idx in sorted(peaks + troughs):  
                # Проверяем, является ли текущий индекс максимумом или минимумом
                if idx in peaks:
                    if y_data[idx] != last_peak_value and (last_trough_value is None or y_data[idx] != last_trough_value):
                        plt.annotate(f'{int(y_data[idx])}', xy=(x_data[idx], y_data[idx]),
                                     xytext=(0, 10), textcoords='offset points', color='green', fontsize=10, ha='center')
                        last_peak_value = y_data[idx]  # Обновляем последнее аннотированное значение пика
                
                elif idx in troughs:
                    if y_data[idx] != last_trough_value and (last_peak_value is None or y_data[idx] != last_peak_value):
                        plt.annotate(f'{int(y_data[idx])}', xy=(x_data[idx], y_data[idx]),
                                     xytext=(0, -15), textcoords='offset points', color='red', fontsize=10, ha='center')
                        last_trough_value = y_data[idx]  # Обновляем последнее аннотированное значение минимума
            
            # Увеличиваем цветовой индекс
            color_index += 1
        
        # Очищаем списки для новой кампании
        x_data, y_data = [], []  

    # Добавляем данные для текущей кампании
    x_data.append(row['date_group'])  # 'date_group' - название столбца с датами
    y_data.append(row['cost'])
    prev_campaign = campaign

# Рисуем линию для последней кампании
if len(x_data) > 0 and len(y_data) > 0:
    plt.plot(x_data, y_data, label=prev_campaign, color=colors(color_index), linewidth=2)  # Установите желаемую толщину
    
    # Находим локальные максимумы и минимумы
    peaks, troughs = find_peaks_and_troughs(x_data, y_data)
    logging.info(f"Campaign: {prev_campaign}, Peaks: {peaks}, Troughs: {troughs}")  # Логирование информации

    # Аннотируем локальные максимумы и минимумы
    last_peak_value = None  # Храним последнее аннотированное значение пика
    last_trough_value = None  # Храним последнее аннотированное значение минимума
    
    # Объединяем индексы пиков и низин
    for idx in sorted(peaks + troughs):  
        # Проверяем, является ли текущий индекс максимумом или минимумом
        if idx in peaks:
            if y_data[idx] != last_peak_value and (last_trough_value is None or y_data[idx] != last_trough_value):
                plt.annotate(f'{int(y_data[idx])}', xy=(x_data[idx], y_data[idx]),
                             xytext=(0, 10), textcoords='offset points', color='green', fontsize=10, ha='center')
                last_peak_value = y_data[idx]  # Обновляем последнее аннотированное значение пика
        
        elif idx in troughs:
            if y_data[idx] != last_trough_value and (last_peak_value is None or y_data[idx] != last_peak_value):
                plt.annotate(f'{int(y_data[idx])}', xy=(x_data[idx], y_data[idx]),
                             xytext=(0, -15), textcoords='offset points', color='red', fontsize=10, ha='center')
                last_trough_value = y_data[idx]  # Обновляем последнее аннотированное значение минимума

# Настройки графика
plt.title('Стоимость рекламы по кампаниям (с изменением цвета)')
plt.xlabel('Дата')
plt.ylabel('Стоимость')

# Форматируем даты на оси X
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))  # Установка интервала для меток на оси X
plt.gcf().autofmt_xdate()  # Автоматическая ориентация меток по оси X

plt.grid(axis='y')
plt.legend(title='Рекламные кампании')

# Сохранение графика
save_plot(plt, 'Стоимости реклам.png')






# # 8. Визиты за весь период с цветовым выделением рекламной кампании (график со столбиками)
# Фильтруем уникальные кампании
filtered_campaigns = ads_summary_df[ads_summary_df['utm_campaign'] != 'none']['utm_campaign'].unique()
overall_max = summary_df['visits'].max()
# Создание графика
plt.figure(figsize=(26, 13))

# Обработка каждой кампании
for campaign in filtered_campaigns:
    campaign_data = summary_df[ads_summary_df['utm_campaign'] == campaign]
    
    # Сортировка данных по дате
    campaign_data = campaign_data.sort_values('date_group')

    # Генерация цвета для области под линией
    color = plt.cm.tab10(filtered_campaigns.tolist().index(campaign) % 10)  # Цвет на основе индекса кампании

    # Закрашиваем область по оси Y от 0 до максимума для текущей кампании
    plt.fill_between(campaign_data['date_group'], 0, overall_max, alpha=0.3, color=color, label=campaign)

    # Строим линейный график
    plt.plot(campaign_data['date_group'], campaign_data['visits'], marker='o', color='black')

plt.title('Визиты за весь период с выделением рекламной кампании')
plt.xlabel('Дата')
plt.ylabel('Количество визитов')
plt.xticks(rotation=45)  # Поворачиваем метки по оси x для лучшей читаемости
plt.grid(axis='y')
plt.grid(axis='x')

# Убираем дублирующие метки в легенде
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.tight_layout()  # Автоматическая подгонка для улучшения отображения

# Сохранение графика
save_plot(plt, 'Визиты за весь период с цветовым выделением рекламной кампании.png')



# # 9. Регистрации за весь период с цветовым выделением рекламной кампании (график со столбиками)
# Фильтруем уникальные кампании
filtered_campaigns = ads_summary_df[ads_summary_df['utm_campaign'] != 'none']['utm_campaign'].unique()
overall_max = summary_df['registrations'].max()
# Создание графика
plt.figure(figsize=(26, 13))

# Обработка каждой кампании
for campaign in filtered_campaigns:
    campaign_data = summary_df[ads_summary_df['utm_campaign'] == campaign]
    
    # Сортировка данных по дате
    campaign_data = campaign_data.sort_values('date_group')

    # Генерация цвета для области под линией
    color = plt.cm.tab10(filtered_campaigns.tolist().index(campaign) % 10)  # Цвет на основе индекса кампании

    # Закрашиваем область по оси Y от 0 до общего максимума для текущей кампании
    plt.fill_between(campaign_data['date_group'], 0, overall_max, alpha=0.3, color=color, label=campaign)

    # Строим линейный график
    plt.plot(campaign_data['date_group'], campaign_data['registrations'], marker='o', color='black')

# Убираем дублирующие метки в легенде
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.title('Регистрации за весь период с выделением рекламной кампании')
plt.xlabel('Дата')
plt.ylabel('Количество регистраций')
plt.xticks(rotation=45)  # Поворачиваем метки по оси x для лучшей читаемости
plt.grid(axis='y')
plt.tight_layout()  # Автоматическая подгонка для улучшения отображения

# Сохранение графика
save_plot(plt, 'Регистрации за весь период с цветовым выделением рекламной кампании.png')